In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functions_to_optimize import f_rastrigin
from CMA_obj import CMA_opt
from PEPG_obj import PEPG_opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SPSA_obj import SPSA_opt
from ADAM_opt import AdamOptimizer
from PSO_obj import PSO_opt

In [2]:
#test rastrigin function 
lower_bound = -1
upper_bound = 1
x_1 = np.linspace(lower_bound,upper_bound,200)[:,np.newaxis]
x_2 = np.linspace(lower_bound,upper_bound,200)[:,np.newaxis]

X_1, X_2 = np.meshgrid(x_1,x_2)

X = np.concatenate([X_1.ravel()[:,np.newaxis] , X_2.ravel()[:,np.newaxis]],axis=1)
F = np.zeros([X.shape[0],1])

for i in range(X.shape[0]):
    F[i] = f_rastrigin(X[i,:])
F_2 = np.reshape(F,[len(x_1),len(x_2)])

F_2 = np.reshape(np.apply_along_axis(f_rastrigin, 1, X),newshape=[len(x_1),len(x_2)])
# plot using plotly
fig = make_subplots(rows=1, cols=1, subplot_titles=["Rastrigin function"])

fig.add_trace(go.Contour(z=F_2, x=X_1[0, :], y=X_2[:, 0], colorscale='Turbo', contours_coloring='fill',  # Fill the contours
                                line_width=1.5, showscale=True), row=1, col=1)
fig.update_layout(height = 500,width = 500)
fig.show()


In [ ]:
#use CMA-ES to optimize rastrigin function
epochs = 3000
N_dim = 1000
pop_size = 100

#init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, 1) + lower_bound
init_pos = 1 * np.ones([N_dim, 1])

CMA_optimizer = CMA_opt(N_dim, pop_size, select_pop=int(pop_size/2), sigma_init=0.5, mean_init=init_pos)
#CMA_optimizer.eigen_update_frequency = 1
#print('Initial position: ', init_pos)
#print rastrigin function value at initial position
print('Initial reward: ', f_rastrigin(init_pos))

best_reward_cma = np.zeros(epochs)

#create empty arrays to store rewards and coordinates at each iteration
rewards_cma = np.zeros([epochs, pop_size])
coordinates_cma = np.zeros([epochs, N_dim, pop_size])

for i in range(int(epochs)):
    #ask for new coordinates
    coordinates = CMA_optimizer.ask()
    #compute rewards / value of function to optimize
    rewards = np.reshape(np.apply_along_axis(f_rastrigin, 1, coordinates.T), newshape=[np.shape(coordinates)[1], 1])
    # give rewards to optimizer for adaptation and mutation 
    CMA_optimizer.tell(reward_table=rewards)
    
    best_reward_cma[i] = np.min(rewards)
    #save history of rewards and coordinates
    rewards_cma[i, :] = rewards.T
    coordinates_cma[i, :, :] = coordinates
    
    if i%100==0:
        print('Best reward at iteration {}: {}'.format(i, np.min(rewards)))
        
# Plotting after all iterations
plt.loglog(best_reward_cma)
plt.xlabel('Iteration')
plt.ylabel('Best Reward')
plt.title('CMA-ES Optimization on Rastrigin Function')
plt.grid(True)
plt.show()


In [ ]:
# use PEPG to optimize rastrigin function
epochs = 3000
N_dim = 1000
pop_size = 100

#create empty arrays to store rewards and coordinates at each iteration
rewards_pepg = np.zeros([epochs, pop_size])
coordinates_pepg = np.zeros([epochs, N_dim, pop_size])


#init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, 1) + lower_bound
init_pos = 1 * np.ones([N_dim, 1])

best_reward_pepg = np.zeros(epochs)

PEPG_optimizer = PEPG_opt(N_dim, pop_size, learning_rate=0.05, starting_mu=init_pos ,starting_sigma=0.5)

for i in range(epochs):
    
    coordinates = PEPG_optimizer.ask()
    coordinates = coordinates.T
    rewards = np.reshape(np.apply_along_axis(f_rastrigin, 1, coordinates.T), newshape=[np.shape(coordinates)[1], 1])
    rewards = rewards
    PEPG_optimizer.tell(rewards)
    
    best_reward_pepg[i] = np.min(rewards)
    #save history of rewards and coordinates
    rewards_pepg[i, :] = rewards.T
    coordinates_pepg[i, :, :] = coordinates
    
    if i%100==0:
        print('Best reward at iteration {}: {}'.format(i, np.min(rewards)))
        
# Plotting after all iterations
plt.loglog(best_reward_pepg)
plt.xlabel('Iteration')
plt.ylabel('Best Reward')
plt.title('PEPG Optimization on Rastrigin Function')
plt.grid(True)
plt.show()


In [ ]:
#use SPSA to optimize rastrigin function
epochs = 16000
N_dim = 1000
pop_size = 100

init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, 1) + lower_bound
init_pos = 1 * np.ones([N_dim, 1])

SPSA_optimizer = SPSA_opt(init_pos,alpha=1e-3,epsilon=0.1)
Adam = AdamOptimizer(init_pos, lr=3e-3, beta1=0.99, beta2=0.999, epsilon=1e-8)
#print('Initial position: ', init_pos)
#print rastrigin function value at initial position
print('Initial reward: ', f_rastrigin(init_pos))

best_reward_spsa = np.zeros(epochs)

#create empty arrays to store rewards and coordinates at each iteration
rewards_spsa = np.zeros([epochs, pop_size])
coordinates_spsa = np.zeros([epochs, N_dim])

for i in range(int(epochs)):
    #perturb parameters
    params_plus, params_minus = SPSA_optimizer.perturb_parameters() 
    #compute rewards / value of function to optimize
    reward_plus = np.apply_along_axis(f_rastrigin, 0, params_plus)
    reward_minus = np.apply_along_axis(f_rastrigin, 0, params_minus)

    # give rewards to optimizer for adaptation and mutation 
    grad = SPSA_optimizer.approximate_gradient(reward_plus ,reward_minus)
    step = Adam.step(grad)
    
    #coordinates_spsa[i, :] = SPSA_optimizer.update_parameters(grad).squeeze()
    coordinates_spsa[i, :] = SPSA_optimizer.update_parameters_step(step).squeeze()
    
    best_reward_spsa[i] = np.apply_along_axis(f_rastrigin, 0, SPSA_optimizer.params)[0]
    #save history of rewards and coordinates
    
    if i%100==0:
        print('Best reward at iteration {}: {}'.format(i, best_reward_spsa[i]))
        
# Plotting after all iterations
plt.loglog(best_reward_spsa)
plt.xlabel('Iteration')
plt.ylabel('Best Reward')
plt.title('SPSA Optimization on Rastrigin Function')
plt.grid(True)
plt.show()


In [3]:
# use PSO to optimize rastrigin function
epochs = 1000
N_dim = 10
pop_size = 55

#create empty arrays to store rewards and coordinates at each iteration
rewards_pso = np.zeros([epochs, pop_size])
coordinates_pso = np.zeros([epochs, N_dim, pop_size])


init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, pop_size) + lower_bound
#init_pos = 1 * np.ones([N_dim, pop_size])

#V_init = 0.1 * np.ones([N_dim, pop_size])
#random initial velocity
V_init = 0.1 * np.random.rand(N_dim, pop_size)

#params dictionary
params = {'c_1': 2.5, 
          'c_2': 0.5,
          'w': 0.9,
          'Vmax': 0.1*(upper_bound-lower_bound),
          'upper_bound': upper_bound,
          'lower_bound': lower_bound,
          }

best_reward_pso = np.zeros(epochs)

PSO_optimizer = PSO_opt(X_init = init_pos,V_init = V_init,params=params)
PSO_optimizer.reward_history = np.zeros([epochs, pop_size])

for i in range(epochs):
    
    coordinates = PSO_optimizer.ask()

    rewards = np.reshape(np.apply_along_axis(f_rastrigin, 1, coordinates.T), newshape=[np.shape(coordinates)[1], 1])
    PSO_optimizer.reward_history[i, :] = rewards.T
    PSO_optimizer.tell(rewards)
    
    best_reward_pso[i] = np.min(rewards)
    #save history of rewards and coordinates
    rewards_pso[i, :] = rewards.T
    coordinates_pso[i, :, :] = coordinates
    
    if i%100==0:
        print('Best reward at iteration {}: {}'.format(i, np.min(rewards)))
        
# Plotting after all iterations
plt.loglog(best_reward_pepg)
plt.xlabel('Iteration')
plt.ylabel('Best Reward')
plt.title('PEPG Optimization on Rastrigin Function')
plt.grid(True)
plt.show()


Best reward at iteration 0: 93.87447572609453
Best reward at iteration 100: 64.06075602079233
Best reward at iteration 200: 59.56800503691502
Best reward at iteration 300: 68.50519060615434
Best reward at iteration 400: 69.66658134925189
Best reward at iteration 500: 63.232737318777
Best reward at iteration 600: 36.373588666404785
Best reward at iteration 700: 65.06824048091579
Best reward at iteration 800: 59.294034327335815
Best reward at iteration 900: 56.58089045946425


NameError: name 'best_reward_pepg' is not defined

In [ ]:
#plot the history of the distribution of the coordinates on top of the contour plot of the rastrigin function
#each subplot is a different epoch
#pepg with 6 subplots
fig, ax = plt.subplots(2, 3, figsize=(10, 6))
fig.suptitle('PEPG Optimization on Rastrigin Function')
epochs_to_plot = [0, 100, 500, 1000, 2000, 2500]
k=0
for i in range(2):
    for j in range(3):
        #plot contour of rastrigin function with jet colormap
        ax[i, j].contourf(X_1, X_2, F_2, cmap='turbo')
        #plot distribution of coordinates with transparency
        ax[i, j].scatter(coordinates_pepg[epochs_to_plot[k], :, 5], coordinates_pepg[epochs_to_plot[k], :, 1], color='darkgreen', alpha=0.2)
        #plot the mean of the distribution
        ax[i, j].scatter(np.mean(coordinates_pepg[epochs_to_plot[k], :, 5]), np.mean(coordinates_pepg[epochs_to_plot[k], :, 1]), color='k')
        ax[i, j].set_title(f'Epoch {epochs_to_plot[k]}')
        ax[i, j].set_xlim([lower_bound, upper_bound])
        ax[i, j].set_ylim([lower_bound, upper_bound])
        k+=1

plt.rcParams['figure.dpi'] = 500 

plt.show()


In [ ]:
#plot the history of the distribution of the coordinates on top of the contour plot of the rastrigin function
#each subplot is a different epoch
#CMA with 6 subplots
fig, ax = plt.subplots(2, 3, figsize=(10, 3))
fig.suptitle('CMA-ES Optimization on Rastrigin Function')
epochs_to_plot = [0, 100, 500, 1000, 2000, 2500]
k=0
for i in range(2):
    for j in range(3):
        #plot contour of rastrigin function with jet colormap
        ax[i, j].contourf(X_1, X_2, F_2, cmap='turbo')
        #plot distribution of coordinates with transparency
        ax[i, j].scatter(coordinates_cma[epochs_to_plot[k], :, 5], coordinates_cma[epochs_to_plot[k], :, 1], color='darkgreen', alpha=0.2)
        #plot the mean of the distribution
        ax[i, j].scatter(np.mean(coordinates_cma[epochs_to_plot[k], :, 5]), np.mean(coordinates_cma[epochs_to_plot[k], :, 1]), color='k')
        ax[i, j].set_title(f'Epoch {epochs_to_plot[k]}')
        ax[i, j].set_xlim([lower_bound, upper_bound])
        ax[i, j].set_ylim([lower_bound, upper_bound])
        k+=1

plt.rcParams['figure.dpi'] = 1000 

plt.show()


In [ ]:

cma_time = 113.5
pepg_time = 10.6
spsa_time = 4.2

fig = make_subplots(rows=1, cols = 2,shared_yaxes=True, subplot_titles=["Performance vs epochs", "Performance vs runtime"])


fig.add_trace(go.Scatter(x=np.arange(3000), y=best_reward_cma, mode='lines', name='CMA-ES',showlegend = False,
                         line=dict(color='coral', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(3000), y=best_reward_pepg, mode='lines', name='PEPG',showlegend=False,
                         line=dict(color='forestgreen', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(16000), y=best_reward_spsa, mode='lines', name='SPSA',showlegend= False,
                         line=dict(color='royalblue', width=2)), row=1, col=1)

fig.update_xaxes(title_text="Epochs",type = 'log', row=1, col=1)
fig.update_yaxes(title_text="f(x)", type = 'log',row=1, col=1)
#update yaxis tick format
fig.update_yaxes(exponentformat="power", row=1, col=1)


fig.add_trace(go.Scatter(x=np.linspace(0,cma_time,len(best_reward_cma)), y=best_reward_cma, mode='lines', name='CMA-ES',
                         line=dict(color='coral', width=2)), row=1, col=2)
fig.add_trace(go.Scatter(x=np.linspace(0,pepg_time,len(best_reward_pepg)), y=best_reward_pepg, mode='lines', name='PEPG',
                         line=dict(color='forestgreen', width=2)), row=1, col=2)
fig.add_trace(go.Scatter(x=np.linspace(0,spsa_time,len(best_reward_spsa)), y=best_reward_spsa, mode='lines', name='SPSA',
                         line=dict(color='royalblue', width=2)), row=1, col=2)
fig.update_xaxes(title_text="Time [s]", type = 'log',row=1, col=2)
fig.update_yaxes( type = 'log',row=1, col=2)

fig.update_layout(height = 400,width = 800,template="plotly_white")

In [ ]:
# this is using plotly it looks much better
epochs_to_plot = [0, 100, 500, 1000, 1500, 2000, 2500,2900]
# Create subplots
fig = make_subplots(rows=2, cols=4, subplot_titles=[f'Epoch {epoch}' for epoch in epochs_to_plot],
                    horizontal_spacing=0.02, vertical_spacing=0.1)

# Index for accessing epochs to plot
k = 0

for i in range(1, 3):
    for j in range(1, 5):
        # Add contour plot of the Rastrigin function
        fig.add_trace(go.Contour(z=F_2, x=X_1[0, :], y=X_2[:, 0], colorscale='Turbo', contours_coloring='fill',  # Fill the contours
                                line_width=0, showscale=False, 
                                 name='', hoverinfo='skip'), row=i, col=j)
        
        # Add scatter plot for the distribution of coordinates
        fig.add_trace(go.Scatter(x=coordinates_cma[epochs_to_plot[k], 0, :], 
                                 y=coordinates_cma[epochs_to_plot[k], 1, :], mode='markers',
                                 marker=dict(color='white', size=5, opacity=0.8, line=dict(color='black', width=1)),
                                 name='Population',showlegend=(k==0)), row=i, col=j)
        
        # Add scatter plot for the mean of the distribution
        fig.add_trace(go.Scatter(x=[np.mean(coordinates_cma[epochs_to_plot[k], 0, :])], 
                                 y=[np.mean(coordinates_cma[epochs_to_plot[k], 1, :])], mode='markers',
                                 marker=dict(color='darkred', size=10, line=dict(color='white', width=1)),
                                 name='Mean',showlegend=(k==0)), row=i, col=j)
        
        # Set axis limits
        fig.update_xaxes(range=[lower_bound, upper_bound], row=i, col=j)
        fig.update_yaxes(range=[lower_bound, upper_bound], row=i, col=j)
        
        k += 1

# Update layout
fig.update_layout(title_text='CMA-ES Optimization on Rastrigin Function', height=500, width=900)

fig.show()

In [ ]:
# this is using plotly it looks much better

epochs_to_plot = [0, 100, 500, 1000, 1500, 2000, 2500,2900]
# Create subplots
fig = make_subplots(rows=2, cols=4, subplot_titles=[f'Epoch {epoch}' for epoch in epochs_to_plot],
                    horizontal_spacing=0.02, vertical_spacing=0.1)

# Index for accessing epochs to plot
k = 0

for i in range(1, 3):
    for j in range(1, 5):
        # Add contour plot of the Rastrigin function
        fig.add_trace(go.Contour(z=F_2, x=X_1[0, :], y=X_2[:, 0], colorscale='Turbo', contours_coloring='fill',  # Fill the contours
                                line_width=0, showscale=False, 
                                 name='', hoverinfo='skip'), row=i, col=j)
        
        # Add scatter plot for the distribution of coordinates
        fig.add_trace(go.Scatter(x=coordinates_pepg[epochs_to_plot[k], 0, :], 
                                 y=coordinates_pepg[epochs_to_plot[k], 1, :], mode='markers',
                                 marker=dict(color='white', size=5, opacity=0.8, line=dict(color='black', width=1)),
                                 name='Population',showlegend=(k==0)), row=i, col=j)
        
        # Add scatter plot for the mean of the distribution
        fig.add_trace(go.Scatter(x=[np.mean(coordinates_pepg[epochs_to_plot[k], 0, :])], 
                                 y=[np.mean(coordinates_pepg[epochs_to_plot[k], 1, :])], mode='markers',
                                 marker=dict(color='darkred', size=10, line=dict(color='white', width=1)),
                                 name='Mean',showlegend=(k==0)), row=i, col=j)
        
        # Set axis limits
        fig.update_xaxes(range=[lower_bound, upper_bound], row=i, col=j)
        fig.update_yaxes(range=[lower_bound, upper_bound], row=i, col=j)
        
        k += 1

# Update layout
fig.update_layout(title_text='PEPG Optimization on Rastrigin Function', height=500, width=900)

fig.show()